In [8]:
from binance.client import Client
import pandas as pd
import time

In [9]:
api_key = "b6ba865a8a866e27a29769183953ae47762ea98861118d3e67c0ea0f52f04488"
secret_key = "af692e426d8f9a27a83c7daeea25a81da4270e22bb8337eb7e858488738babd3"

In [10]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True)

In [12]:
client.futures_account()

{'feeTier': 0,
 'canTrade': True,
 'canDeposit': True,
 'canWithdraw': True,
 'tradeGroupId': -1,
 'updateTime': 0,
 'multiAssetsMargin': False,
 'totalInitialMargin': '45.87924393',
 'totalMaintMargin': '1.83516976',
 'totalWalletBalance': '13789.40247515',
 'totalUnrealizedProfit': '25.74489383',
 'totalMarginBalance': '13815.14736898',
 'totalPositionInitialMargin': '45.87924393',
 'totalOpenOrderInitialMargin': '0.00000000',
 'totalCrossWalletBalance': '13741.50140983',
 'totalCrossUnPnl': '0.00000000',
 'availableBalance': '13741.50140983',
 'maxWithdrawAmount': '13741.50140983',
 'assets': [{'asset': 'ETH',
   'walletBalance': '0.00000000',
   'unrealizedProfit': '0.00000000',
   'marginBalance': '0.00000000',
   'maintMargin': '0.00000000',
   'initialMargin': '0.00000000',
   'positionInitialMargin': '0.00000000',
   'openOrderInitialMargin': '0.00000000',
   'maxWithdrawAmount': '0.00000000',
   'crossWalletBalance': '0.00000000',
   'crossUnPnl': '0.00000000',
   'availableBa

In [13]:
def is_bullish_sma_crossover(symbol, interval='1h', short_window=50, long_window=200):
    # Fetch candlestick data
    bars = client.futures_historical_klines(symbol=symbol, interval=interval,start_str=None)
    df = pd.DataFrame(bars)

    # Print bars for debugging
    print('bars:', bars)

    # Extract relevant data and create a DataFrame
    df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Calculate short-term and long-term SMAs
    df['short_sma'] = df['close'].rolling(window=short_window, min_periods=1).mean()
    df['long_sma'] = df['close'].rolling(window=long_window, min_periods=1).mean()

    # Check for bullish crossover
    if df['short_sma'].iloc[-1] > df['long_sma'].iloc[-1] and df['short_sma'].iloc[-2] <= df['long_sma'].iloc[-2]:
        return True
    else:
        return False


In [14]:
def place_market_buy_order(symbol, quantity):
    order = client.futures_create_order(
        symbol=symbol,
        side='BUY',
        type='MARKET',
        quantity=quantity
    )
    print('Buy Order:', order)

def place_market_sell_order(symbol, quantity):
    order = client.futures_create_order(
        symbol=symbol,
        side='SELL',
        type='MARKET',
        quantity=quantity
    )
    print('Sell Order:', order)


In [15]:
symbol = 'BTCUSDT'
quantity = 1  # Adjust based on your risk tolerance

while True:
    if is_bullish_sma_crossover(symbol):
        place_market_buy_order(symbol, quantity)
    elif is_bearish_sma_crossover(symbol):
        place_market_sell_order(symbol, quantity)

    # Adjust the sleep duration based on your desired frequency
    time.sleep(300)  # Wait for 5 minutes before checking again


bars: [[1705654800000, '42497.00', '42499.90', '42130.00', '42140.00', '4212.838', 1705658399999, '178316933.04820', 5163, '2142.004', '91012287.62190', '0'], [1705658400000, '42450.00', '42500.00', '42140.00', '42456.00', '3928.810', 1705661999999, '166262560.99060', 5284, '1957.754', '83195826.87740', '0'], [1705662000000, '42140.00', '42499.90', '42130.00', '42489.00', '3849.969', 1705665599999, '162945939.98430', 4832, '1921.521', '81649786.25810', '0'], [1705665600000, '42489.00', '42499.50', '42140.10', '42339.90', '7065.609', 1705669199999, '298484523.16700', 5112, '2260.557', '95877862.90790', '0'], [1705669200000, '42339.90', '42499.00', '42140.10', '42140.10', '4837.985', 1705672799999, '204488343.26480', 4486, '2083.450', '88339192.25640', '0'], [1705672800000, '42140.10', '42490.00', '42140.10', '42141.00', '4415.514', 1705676399999, '186560263.80860', 4624, '1878.981', '79662855.34600', '0'], [1705676400000, '42141.00', '42498.80', '42131.00', '42449.90', '5862.303', 17056

ValueError: 6 columns passed, passed data had 12 columns